In [1]:
import pandas as pd
import numpy as np
Emotion=['sick','sick','sick','notsick','notsick','sick','notsick','notsick']
Temperature = ['under','over','under','under','over','over','under','over']
StayHome=['N','Y','Y','N','Y','N','N','Y']
df=pd.DataFrame (list(zip(Emotion,Temperature,StayHome)),
                columns=["Emotion","Temperature","StayHome"])
print(df.sort_values(['Temperature','StayHome']).reset_index(drop=True))

   Emotion Temperature StayHome
0     sick        over        N
1     sick        over        Y
2  notsick        over        Y
3  notsick        over        Y
4     sick       under        N
5  notsick       under        N
6  notsick       under        N
7     sick       under        Y


In [3]:

# Identify the Outcome Variable
Outcome='StayHome'
# Creates a list of index without the outcome column
outcome=df.columns.get_loc(Outcome)
col_ls=list(range(len(df.columns)))
col_ls.remove(outcome)
#print feature positions
col_ls

[0, 1]

In [4]:
# create copy of data and get unique values in each feature
new_df = df
names, count=np.unique(new_df.iloc[:,col_ls[0]],return_counts=True)
# returns how many unique feature there are in this split
splitdf=len(names)
for i in range(0,len(names)):
    print("We have split on feature:","(",new_df.columns[0],")","where it equals","(",names[i],")","and has",count[i],"observations \n")

We have split on feature: ( Emotion ) where it equals ( notsick ) and has 4 observations 

We have split on feature: ( Emotion ) where it equals ( sick ) and has 4 observations 



In [5]:
for n in names:
    print("-----------------------------------------")
    print(new_df[new_df.iloc[:,col_ls[0]]==n],"\n","\n")

-----------------------------------------
   Emotion Temperature StayHome
3  notsick       under        N
4  notsick        over        Y
6  notsick       under        N
7  notsick        over        Y 
 

-----------------------------------------
  Emotion Temperature StayHome
0    sick       under        N
1    sick        over        Y
2    sick       under        Y
5    sick        over        N 
 



In [7]:

splitdata=new_df[new_df.iloc[:,col_ls[0]]==names[1]]
# outcome is defined above

#gets the unique outcome values
value=np.unique(df.iloc[:,outcome])
#This will be used later when we computed the weighted gini value
denS=len(splitdata)
impurity=0
                 
for values in value:
            p=splitdata.iloc[:,outcome].eq(values).sum()/denS
            impurity +=p*(1-p)
print ("The gini impurity index is: ",impurity,"for \n \n", splitdata.sort_values("StayHome").reset_index(drop=True))

The gini impurity index is:  0.5 for 
 
   Emotion Temperature StayHome
0    sick       under        N
1    sick        over        N
2    sick        over        Y
3    sick       under        Y


In [8]:
#create empty list to hold each splits impurity and split count
gini_metric=list()
den_split=list()
        
for n in names:
    splitdata=new_df[new_df.iloc[:,col_ls[0]]==n]
    value=np.unique(df.iloc[:,outcome])
#This will be used later when we computed the weighted gini value
    denS=len(splitdata)
    impurity=0
                 
    for values in value:
            p=splitdata.iloc[:,outcome].eq(values).sum()/denS
            impurity +=p*(1-p)
    gini_metric.append(impurity)
    den_split.append(denS)


feature=splitdf
# get the proporation of the split for current feature
den_split1=[x/sum(den_split[0:feature]) for x in den_split[0:feature]]
weighted_gini=sum(np.asarray(den_split1)*np.asarray(gini_metric[0:feature]))

print("The weighted gini impurtiy measure is:",weighted_gini)

The weighted gini impurtiy measure is: 0.5


In [9]:
# Generalize our gini Function
def Gini(splitdata,outcome):
    # Get unique Outcome values
    value=np.unique(splitdata.iloc[:,outcome])
    denS=len(splitdata)
    impurity=0
    for values in value:
                    p=splitdata.iloc[:,outcome].eq(values).sum()/denS
                    impurity +=p*(1-p)
    return (impurity,denS)

#Tree
def SimpleTree(df,Outcome):
    if type(Outcome)==str:
        
         #empty list to hold information
        splitdf=list()
        gini_metric=list()
        w_gini=list()
        den_split=list()
        split_data=list()
        count=0
        split_data_index=list()
        P=list()
        
        #Creates a list of index without the outcome column
        outcome=df.columns.get_loc(Outcome)
        col_ls=list(range(len(df.columns)))
        col_ls.remove(outcome)
       
        #begin loops
        for col in col_ls:
            #create copy of data and get unique values in each feature
            new_df=df
            names,count=np.unique(new_df.iloc[:,col],return_counts=True)
            #Returns the number of splits per feature 
            splitdf.append(len(names))
            
            for n in names:
                #returns the columns such that it equals the split criteria
                split_df=new_df[new_df.iloc[:,col]==n]
                #add df to list
                split_data.append(split_df)
                #keep track of the columns split data set
                index=pd.DataFrame(list([col]))
                split_data_index.append(index)
     
        # Get unique Outcome values
        value=np.unique(df.iloc[:,outcome])
        
        for data in split_data:
            #calcualte Gini at each split
            impurity,denS=Gini(data,outcome)
            #append lists
            gini_metric.append(impurity)
            den_split.append(denS)
        # For loop which will iterate on the number of splits per feature to get the WEIGHTED GINI METRIC
        for feature in splitdf:
            # get the proporation of the split for current feature
            den_split1=[x/sum(den_split[0:feature]) for x in den_split[0:feature]]
            weighted_gini=sum(np.asarray(den_split1)*np.asarray(gini_metric[0:feature]))
            #remove the gini and feature count split for the next iteration
                              
            w_gini.append(weighted_gini)
            del(gini_metric[0:feature], den_split[0:feature])
    else: print("Place Outcome Variable Name in \'Single\' quotes")
    #return feature with the smallest impurity index
    f_col=w_gini.index(min(w_gini))
    #returns the name
    bestsplit=df.columns[f_col]
    return(col_ls,w_gini,split_data,split_data_index,f_col)
#------------------------------------------------------------------------------------------

In [10]:
col_ls,w_gini,split_data,split_index,bestsplit=SimpleTree(df,"StayHome")
for col in col_ls:
           print("Feature","(",df.columns[col],")","Weighted Gini Impurity Index is:", w_gini[col],"\n")
print("The Best split is with feature","(",df.columns[bestsplit],")")

Feature ( Emotion ) Weighted Gini Impurity Index is: 0.5 

Feature ( Temperature ) Weighted Gini Impurity Index is: 0.375 

The Best split is with feature ( Temperature )
